
# Parcial 2: El modelo VGG16

Se decidió utilizar VGG16, un modelo de red neuronal convolucional profundo propuesto por [Karen Simonyan][karen] y [Andrew Zisserman][andrew]  en [*Very Deep Convolutional Networks for Large-Scale Image Recognition*][paper]. 

El proyecto de investigación fué realizado a través del Visual Geometry Group -*VGG*- del Depto. de Ingeniería de la Universidad de Oxford y es por esta razón que fué bautizada con su acrónimo. El sufijo "*16*" indica la cantidad de capas que posee el modelo.

[karen]: https://scholar.google.com/citations?user=L7lMQkQAAAAJ&hl=en
[andrew]: https://scholar.google.com/citations?user=UZ5wscMAAAAJ&hl=en
[paper]:https://arxiv.org/pdf/1409.1556.pdf

## Introducción

Este modelo fué uno de los más populares entre los postulados al [ILSVRC-14](https://www.image-net.org/challenges/LSVRC/2014/) alcanzando un 92.7% de precisión gracias a la introducción de múltiples filtros con tamaño de kernel 3x3 uno detrás de otro.

## ¿Cómo funciona?
El modelo VGG16 posee la siguiente arquitectura:

![](https://neurohive.io/wp-content/uploads/2018/11/vgg16-1-e1542731207177.png)


La red se alimenta de imágenes de 224x224x3 píxeles (es decir imágenes a color de 224x224 píxeles de resolución) y posee una serie de capas convolucionales, cuya función de rectificación es ReLU no-lineal, donde cada par o tríada es seguido por una respectiva capa de Max-pooling cuya ventana es de 2x2 pixeles con *stride*=2. Por último, posee tres capas densas -completamente conexas- también con ReLU no-lineal como función de activación. Las primeras dos son de tamaño 1x4096 mientras que la última posee un tamaño de 1x1000 clases y actúa como capa de clasificación aplicando *softmax*.

![](https://neurohive.io/wp-content/uploads/2018/11/vgg16.png)


La capa final clasificadora tiene su razón de ser justificada en el funcionamiento del desafío de ImageNet [ILSVRC-14](https://www.image-net.org/challenges/LSVRC/2014/) donde se proveen varias clases de imágenes, cada una con 1000 imágenes de sample. 

Para mayor detalle, y conocimiento de los demás modelos propuestos por los autores, revisar [*Very Deep Convolutional Networks for Large-Scale Image Recognition*][paper]. 

## Modo de uso

Se optó por una [versión pre-entrenada][keras] e implementada en la librería Keras de Python, permitiendo ahorrar todo el tiempo de procesamiento que demandaría entrenarla por la gran profundidad que posee.

Los pesos utilizados también serán provistos por la librería Keras, cuando el modelo se instancie se descargarán automáticamente.


[keras]:https://keras.io/api/applications/vgg/

## Testeo

Para corroborar el funcionamiento y grado de precisión con el que clasifica este modelo se lo alimentará con 25 imágenes (de diferentes resoluciones y formatos) que serán re-escaladas al tamaño esperado por la red 224x224. 

Se buscó que las imágenes sean de diferentes objetos, paisajes o personas para ver qué interpretaba el clasificador.

A continuación, se procede con la instanciación del modelo y su posterior clasificación. Finalmente los resultados se podrán ver en formato tabular (y también se descargarán en formato .csv)

In [1]:
# Descargo todas las imagenes a clasificar
!wget https://raw.githubusercontent.com/ncavasin/sistemas_inteligentes/main/parcial_2/vgg/images/imgs_compressed.zip
!unzip imgs_compressed -d images

--2021-06-20 20:24:52--  https://raw.githubusercontent.com/ncavasin/sistemas_inteligentes/main/parcial_2/vgg/images/imgs_compressed.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6205945 (5.9M) [application/zip]
Saving to: ‘imgs_compressed.zip’

imgs_compressed.zip 100%[===================>]   5.92M  --.-KB/s    in 0.1s    

2021-06-20 20:24:52 (39.9 MB/s) - ‘imgs_compressed.zip’ saved [6205945/6205945]

Archive:  imgs_compressed.zip
  inflating: images/01_tomato.jpg    
  inflating: images/02_glasses.jpg   
  inflating: images/03_car.jpg       
  inflating: images/04_person.jpg    
  inflating: images/05_house.jpg     
  inflating: images/06_waves.jpg     
  inflating: images/07_tower_bridge.jpg  
  inflating: images/08_rhino.jpg     
  inflating: imag

In [2]:
# Importo librerias necesarias
import os
import keras
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.vgg16 import decode_predictions as vgg16_decode
from tensorflow.keras.utils import plot_model
import numpy as np
import pandas as pd

In [3]:
## Funciones necesarias

# Adapto la imagen al tamaño que espera la primer capa de la red
def img_to_2darray(img: np.ndarray) -> np.ndarray:
    transformedImage = image.img_to_array(img)
    transformedImage = np.expand_dims(transformedImage, axis = 0)
    return vgg16_preprocess(transformedImage)

# Clasifico todas las imagenes en el directorio indicado
def predict(model: keras.Model, dir:str) -> list:
    lista = []

    for filename in os.listdir(dir):
        img = image.load_img(dir+filename, color_mode='rgb', target_size=(224, 224))

        img = img_to_2darray(img)
        prediction = model.predict(img)
        predicted_class = vgg16_decode(prediction, top = 5)

        top_class_label     = predicted_class[0][0][1]
        top_class_certainty = f'{(predicted_class[0][0][2]*100):.2f}%'
        alt_1 = (predicted_class[0][1][1], f'{(predicted_class[0][1][2]*100):.2f}%')
        alt_2 = (predicted_class[0][2][1], f'{(predicted_class[0][2][2]*100):.2f}%')
        alt_3 = (predicted_class[0][3][1], f'{(predicted_class[0][3][2]*100):.2f}%')
        alt_4 = (predicted_class[0][4][1], f'{(predicted_class[0][4][2]*100):.2f}%')

        name, ext = filename.split('.')
        lista.append([name, top_class_label, 
                      top_class_certainty, 
                      alt_1[0], alt_1[1], 
                      alt_2[0], alt_2[1], 
                      alt_3[0], alt_3[1],
                      alt_4[0], alt_4[1]])

    return lista

In [4]:
# Instancio el modelo y disparo descarga de sus pesos
model = VGG16(include_top=True, weights='imagenet', classes=1000, classifier_activation='softmax')

# Muestro resumidamente la arquitectura del modelo instanciado
model.summary()

553467904/553467096 [==============================] - 4s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
____________________________________________________________

In [5]:
# Clasifico y disparo descarga de los nombres de cada clase para poder indexar
predicciones = predict(model, 'images/')

40960/35363 [==================================] - 0s 0us/step


In [6]:
# Ordeno las predicciones por nombre, guardo a csv y muestro en formato tabular
predicciones.sort()
df = pd.DataFrame(predicciones, columns=['archivo','clase','acc','clase 2','acc','clase 3','acc','clase 4','acc','clase 5','acc'])
df.to_csv('predicciones.csv')
df 

# Descomentar para generar nuevamente la imagen de la arquitectura instanciada
#plot_model(model, to_file='vgg_model.png', show_shapes=True, show_dtype=True, show_layer_names=True, rankdir='TB')

,archivo,clase,acc,clase 2,acc,clase 3,acc,clase 4,acc,clase 5,acc
0,01_tomato,hip,50.60%,bell_pepper,20.95%,strawberry,20.60%,pomegranate,2.19%,pineapple,0.95%
1,02_glasses,sunglasses,48.85%,sunglass,47.76%,loupe,0.57%,hook,0.53%,stethoscope,0.31%
2,03_car,sports_car,83.92%,beach_wagon,8.63%,car_wheel,3.53%,convertible,1.89%,racer,1.43%
3,04_person,torch,48.57%,jersey,5.56%,racket,4.45%,ballplayer,3.75%,lab_coat,3.38%
4,05_house,boathouse,13.59%,mobile_home,11.63%,picket_fence,10.04%,patio,9.11%,lakeside,7.86%
5,06_waves,spindle,15.24%,great_white_shark,6.30%,dam,5.61%,speedboat,4.87%,volcano,4.42%
6,07_tower_bridge,suspension_bridge,68.60%,pier,29.05%,steel_arch_bridge,0.42%,crane,0.27%,castle,0.21%
7,08_rhino,bison,94.79%,water_buffalo,3.50%,ox,0.99%,warthog,0.41%,hyena,0.07%
8,09_downey_jr,suit,46.59%,Windsor_tie,40.16%,bow_tie,4.26%,oboe,0.93%,bolo_tie,0.53%
9,10_table,folding_chair,64.55%,desk,10.71%,dining_table,7.26%,nail,3.94%,pedestal,1.58%
